## scikit-learn sample_weight compliance report

This notebook runs compliance tests on all scikit-learn estimators. Estimator as inspected to check whether they are expected to have a stochastic fit or not. If the fit is stochastic, a dedicated statistical test is performed, otherwise a deterministic estimator check is run instead.

In [1]:
import sklearn

sklearn.show_versions()


System:
    python: 3.12.4 | packaged by conda-forge | (main, Jun 17 2024, 10:13:44) [Clang 16.0.6 ]
executable: /Users/shrutinath/micromamba/envs/scikit-learn/bin/python
   machine: macOS-14.3-arm64-arm-64bit

Python dependencies:
      sklearn: 1.8.dev0
          pip: 24.0
   setuptools: 75.8.0
        numpy: 2.0.0
        scipy: 1.15.3
       Cython: 3.0.10
       pandas: 2.2.2
   matplotlib: 3.9.0
       joblib: 1.4.2
threadpoolctl: 3.5.0

Built with OpenMP: True

threadpoolctl info:
       user_api: blas
   internal_api: openblas
    num_threads: 8
         prefix: libopenblas
       filepath: /Users/shrutinath/micromamba/envs/scikit-learn/lib/libopenblas.0.dylib
        version: 0.3.27
threading_layer: openmp
   architecture: VORTEX

       user_api: openmp
   internal_api: openmp
    num_threads: 8
         prefix: libomp
       filepath: /Users/shrutinath/micromamba/envs/scikit-learn/lib/libomp.dylib
        version: None


In [2]:
from inspect import signature
import traceback
import warnings
import pandas as pd
from sklearn.utils import all_estimators
from sklearn.exceptions import ConvergenceWarning
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import KBinsDiscretizer
import threadpoolctl

from sample_weight_audit import check_weighted_repeated_estimator_fit_equivalence
from sample_weight_audit.sklearn_stochastic_params import STOCHASTIC_FIT_PARAMS

# HistGradientBoostingClassifier trashes the OpenMP thread pool on repeated
# small fits.
threadpoolctl.threadpool_limits(limits=1, user_api="openmp")
warnings.filterwarnings("ignore", category=RuntimeWarning)  # division by zero in AdaBoost
warnings.filterwarnings("ignore", category=ConvergenceWarning)  # liblinear can fail to converge
warnings.filterwarnings("ignore", category=UserWarning)  # KBinsDiscretizer with collapsed bins

In [3]:
from sklearn.linear_model import LogisticRegressionCV

ESTIMATORS_TO_SKIP = [
    LogisticRegressionCV,  # too slow and already somewhat tested by LogisticRegression
]

In [4]:
N_STOCHASTIC_FITS = 100
N_STATISTICAL_TESTS = 58  # measured by running the script
BONFERRONI_CORRECTION = 1 / N_STATISTICAL_TESTS
TEST_THRESHOLD = 0.05 * BONFERRONI_CORRECTION


statistical_test_results = []
missing_sample_weight_support = []
errors = []


for est_name, est_class in all_estimators(
    type_filter=["classifier", "regressor", "cluster", "transformer"]
):
    if est_class in ESTIMATORS_TO_SKIP:
        print(f"Skipping {est_name}")
        continue

    if "sample_weight" not in signature(est_class.fit).parameters:
        print(f"⚠ {est_name} does not support sample_weight")
        missing_sample_weight_support.append(est_name)
        continue

    try:
        est = est_class(**STOCHASTIC_FIT_PARAMS.get(est_class, {}))
    except TypeError as e:
        print(f"⚠ {est_name} failed to instantiate: {e}")
        continue

    print(f"Evaluating {est}")
    try:
        result = check_weighted_repeated_estimator_fit_equivalence(
            est,
            est_name,
            test_name="kstest",
            n_stochastic_fits=N_STOCHASTIC_FITS,
            store_estimators=True,
            random_state=0,
        )
        pass_or_fail = "✅" if result.pvalue > TEST_THRESHOLD else "❌"
        print(f"{pass_or_fail} {est_name}: (pvalue: {result.pvalue:.3f})")
        statistical_test_results.append(result)
    except Exception as e:
        print(f"❌ {est} error with: {e}")
        errors.append((est, e))

results_df = pd.DataFrame([r.to_dict() for r in statistical_test_results])

⚠ ARDRegression does not support sample_weight
Evaluating AdaBoostClassifier(estimator=DecisionTreeClassifier(max_features=0.5,
                                                    min_weight_fraction_leaf=0.1))


100%|██████████| 100/100 [00:05<00:00, 18.81it/s]


✅ AdaBoostClassifier: (pvalue: 0.211)
Evaluating AdaBoostRegressor(estimator=DecisionTreeRegressor(max_features=0.5,
                                                  min_weight_fraction_leaf=0.1))


100%|██████████| 100/100 [00:04<00:00, 22.41it/s]


❌ AdaBoostRegressor: (pvalue: 0.001)
⚠ AdditiveChi2Sampler does not support sample_weight
⚠ AffinityPropagation does not support sample_weight
⚠ AgglomerativeClustering does not support sample_weight
Evaluating BaggingClassifier(estimator=LogisticRegression())


...

✅ SplineTransformer: (pvalue: 1.000)
⚠ StackingClassifier does not support sample_weight
⚠ StackingRegressor does not support sample_weight
Evaluating StandardScaler()


100%|██████████| 1/1 [00:00<00:00, 523.50it/s]


✅ StandardScaler: (pvalue: 1.000)
⚠ TSNE does not support sample_weight
⚠ TargetEncoder does not support sample_weight
⚠ TfidfTransformer does not support sample_weight
⚠ TheilSenRegressor does not support sample_weight
⚠ TransformedTargetRegressor does not support sample_weight
⚠ TruncatedSVD does not support sample_weight
⚠ TunedThresholdClassifierCV does not support sample_weight
Evaluating TweedieRegressor()


100%|██████████| 1/1 [00:00<00:00, 476.19it/s]

✅ TweedieRegressor: (pvalue: 1.000)
⚠ VarianceThreshold does not support sample_weight
⚠ VotingClassifier does not support sample_weight
⚠ VotingRegressor does not support sample_weight


In [5]:
print(
    f"✅ {len([r for r in statistical_test_results if r.pvalue > TEST_THRESHOLD])} "
    "passed the statistical test"
)
print(
    f"❌ {len([r for r in statistical_test_results if r.pvalue <= TEST_THRESHOLD])} "
    "failed the statistical test"
)
print(f"❌ {len(errors)} other errors")
print(
    f"⚠ {len(missing_sample_weight_support)} estimators lack sample_weight "
    "support"
)
results_df = pd.DataFrame([r.to_dict() for r in statistical_test_results])

✅ 45 passed the statistical test
❌ 13 failed the statistical test
❌ 4 other errors
⚠ 116 estimators lack sample_weight support


## Details on the statistical test results

In [6]:
results_df.sort_values("pvalue")[["estimator_name", "pvalue", "deterministic_predictions"]]

,estimator_name,pvalue,deterministic_predictions
24,HistGradientBoostingClassifier,2.208761e-59,False
47,RandomForestRegressor,2.900986e-53,False
46,RandomForestClassifier,1.820209e-48,False
2,BaggingClassifier,2.596277e-44,False
36,LogisticRegression,2.676640e-25,False
41,NuSVC,3.357101e-13,False
3,BaggingRegressor,1.070076e-12,False
43,Perceptron,1.115168e-08,False
51,SGDClassifier,6.617358e-08,False
52,SGDRegressor,7.850159e-06,False


In [12]:
results_df.iloc[0]['estimators_weighted']

[AdaBoostClassifier(estimator=DecisionTreeClassifier(max_features=0.5,
                                                     min_weight_fraction_leaf=0.1),
                    random_state=0),
 AdaBoostClassifier(estimator=DecisionTreeClassifier(max_features=0.5,
                                                     min_weight_fraction_leaf=0.1),
                    random_state=1),
 AdaBoostClassifier(estimator=DecisionTreeClassifier(max_features=0.5,
                                                     min_weight_fraction_leaf=0.1),
                    random_state=2),
 AdaBoostClassifier(estimator=DecisionTreeClassifier(max_features=0.5,
                                                     min_weight_fraction_leaf=0.1),
                    random_state=3),
 AdaBoostClassifier(estimator=DecisionTreeClassifier(max_features=0.5,
                                                     min_weight_fraction_leaf=0.1),
                    random_state=4),
 AdaBoostClassifier(estimator=DecisionTr

## Details on errors

In [7]:
import sys

for est, e in errors:
    print(f"❌ {est}: {e}")
    traceback.print_exception(e, file=sys.stdout)
    print()

❌ CategoricalNB(): index 7 is out of bounds for axis 1 with size 7
Traceback (most recent call last):
  File "/var/folders/h4/l_wtcs4s43j2j9454pvp1c580000gn/T/ipykernel_41249/3518123444.py", line 32, in <module>
    result = check_weighted_repeated_estimator_fit_equivalence(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shrutinath/sample_weight_audit_dist/src/sample_weight_audit/estimator_check.py", line 97, in check_weighted_repeated_estimator_fit_equivalence
    multifit_results = multifit_over_weighted_and_repeated(
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shrutinath/sample_weight_audit_dist/src/sample_weight_audit/estimator_check.py", line 372, in multifit_over_weighted_and_repeated
    scores_weighted, preds_weighted = score_estimator(
                                      ^^^^^^^^^^^^^^^^
  File "/Users/shrutinath/sample_weight_audit_dist/src/sample_weight_audit/estimator_check.py", line 204, in score_estimator
  

## List of estimators with missing sample_weight support

In [8]:
for est_name in missing_sample_weight_support:
    print(est_name)

ARDRegression
AdditiveChi2Sampler
AffinityPropagation
AgglomerativeClustering
BernoulliRBM
Binarizer
Birch
CCA
ClassifierChain
ColumnTransformer
DictVectorizer
DictionaryLearning
FactorAnalysis
FastICA
FeatureAgglomeration
FeatureHasher
FeatureUnion
FixedThresholdClassifier
FunctionTransformer
GaussianProcessClassifier
GaussianProcessRegressor
GaussianRandomProjection
GenericUnivariateSelect
HDBSCAN
HashingVectorizer
IncrementalPCA
Isomap
KNNImputer
KNeighborsClassifier
KNeighborsRegressor
KNeighborsTransformer
KernelCenterer
KernelPCA
LabelBinarizer
LabelEncoder
LabelPropagation
LabelSpreading
Lars
LarsCV
LassoLars
LassoLarsCV
LassoLarsIC
LatentDirichletAllocation
LinearDiscriminantAnalysis
LocallyLinearEmbedding
MaxAbsScaler
MeanShift
MinMaxScaler
MiniBatchDictionaryLearning
MiniBatchNMF
MiniBatchSparsePCA
MissingIndicator
MultiLabelBinarizer
MultiTaskElasticNet
MultiTaskElasticNetCV
MultiTaskLasso
MultiTaskLassoCV
NMF
NearestCentroid
NeighborhoodComponentsAnalysis
Normalizer
Nystroe

## Example interactive inspection of the predictions and scores of a given estimator

In [9]:
results = next(r for r in statistical_test_results if r.estimator_name == "GaussianNB")
results.predictions_repeated.shape

(1, 3000)

In [10]:
results.predictions_weighted.shape

(1, 3000)

In [11]:
import numpy as np
np.abs(results.predictions_repeated - results.predictions_weighted).max()

np.float64(3.9999992207384594e-10)

In [12]:
np.abs(results.scores_repeated - results.scores_weighted).max()

np.float64(3.2652280879119644e-10)

In [13]:
from scipy.stats import kstest
kstest(results.scores_repeated, results.scores_weighted)

KstestResult(statistic=np.float64(1.0), pvalue=np.float64(1.0), statistic_location=np.float64(3.262703314947884), statistic_sign=np.int8(1))